# Exercise 1  - Python example

*Created by Julia Kukulies* 

In [1]:
# import necessary python packages 

import xarray as xr # to access netCDF4 files 
import numpy as np # for calculations and matrix handling 

## Part 1: CMIP6 wind speed from CMCC

In Python, there are multiple ways to read in netCDF files. An alternative package is called **netCDF4** (https://unidata.github.io/netcdf4-python/netCDF4/index.html). However, **xarray** (http://xarray.pydata.org/en/stable/) is a practical data container, since it is easy to access all relevant information for each datapoint. If you do any calculations to the data, each datapoint is still associated with the correct lat, lon and time values. 

In [2]:
# read in netcdf file with xarray 
path = 'sfcWind_Amon_CMCC-CM2-HR4_historical_r1i1p1f1_gn_185001-201412.nc'
dataset= xr.open_dataset(path)
# this is how your data array looks like: 
display(dataset)

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 192, lon: 288, time: 1980)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    sfcWind    (time, lat, lon) float32 ...
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    comment:                none
    contact:                T. Lovato
    creation_date:          2020-08-26T10:13:36Z
    data_specs_version:     01.00.31
    experiment:             all-forcing simulation of the recent past
    experiment_id:          historical
    external_variables:     areacella
    forcing_index:          1
    frequency:              mon
    further_info_url:       https://furtherinfo.es-doc.org/CMIP6.CMCC.CMCC-CM...
    grid:                   native atmosphere regular grid
    grid_label:             gn
    history:                2020-08-26T10:13:36Z ;rewrote data to be consiste...
    initialization_index:   1
    institution:            Fondazione Centro Euro-Mediterraneo sui Cambiamen...
    institution_id:         CMCC
    mip_era:                CMIP6
    nominal_resolution:     100 km
    parent_activity_id:     CMIP
    parent_experiment_id:   piControl
    parent_mip_era:         CMIP6
    parent_source_id:       CMCC-CM2-HR4
    parent_time_units:      days since 1850-01-01
    parent_variant_label:   r1i1p1f1
    physics_index:          1
    product:                model-output
    realization_index:      1
    realm:                  atmos
    references:             none
    run_variant:            1st realization
    source:                 CMCC-CM2-HR4 (2016): \naerosol: prescribed MACv2-...
    source_id:              CMCC-CM2-HR4
    source_type:            AOGCM
    sub_experiment:         none
    sub_experiment_id:      none
    table_id:               Amon
    table_info:             Creation Date:(05 February 2020) MD5:6a248fd76c55...
    title:                  CMCC-CM2-HR4 output prepared for CMIP6
    variable_id:            sfcWind
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by CMCC is licensed und...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/1a57ce91-cee3-40b1-b897-fe676113ec2e

In [3]:
# by calling the attribute values the variables are loaded as numpy arrays
# numpy is a practical data container to work with (especially for any mathematical operations)
longitude = dataset.lon.values
latitude = dataset.lat.values
windspeed = dataset.sfcWind.values

# check out the dimensions of each variables now:
print('longitude:', longitude.shape, 'latitude:', latitude.shape, 'windspeed:', windspeed.shape)

longitude: (288,) latitude: (192,) windspeed: (1980, 192, 288)


**Get a time vector which is understandable for you**


In [4]:
# the time dimension in the dataset looks like this (cftime.DatetimeNoLeap is a specific data type in Python)
# there are many different formats to deal with dates in python (practical for more complicated time operations)
time = dataset.time.values
time

array([cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(1850, 2, 15, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(1850, 3, 16, 12, 0, 0, 0), ...,
       cftime.DatetimeNoLeap(2014, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeNoLeap(2014, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeNoLeap(2014, 12, 16, 12, 0, 0, 0)], dtype=object)

In [5]:
# for simplicity we convert the array to a simple string
# create empty array in which to store years
yyyymm = np.array(())

# access years and months for each element by looping and extracting attributes of datetime objects 
# note that datetime format is handy, because years and months are attributes of the date)
for date in time:
    # year as string 
    year = str(date.year)
    # month as string 
    month = str(date.month)
    # zero-padding for months < 10, to obtain same length
    if date.month < 10:
        month = '0' + str(date.month)
    # add joined string to new array 
    yyyymm = np.append(yyyymm, year+month)

In [6]:
# check now the values you put into your new arrays 
print(yyyymm)

# check how long the arrays are (should be same as first dimension (time) of wind speeds)
print(yyyymm.shape)

['185001' '185002' '185003' ... '201410' '201411' '201412']
(1980,)


**What was the wind speed in the grid cell closest to Gothenburg in April 1935** 

Gothenburg coordinates: 

Lon 12$^\circ$E, Lat 58$^\circ$N

In [7]:
lon = 12
lat = 58

# get index for closest longitude: where is longitude - 12 deg closest to 0? 
lon_idx = (np.abs(longitude - lon)).argmin()
# same for latitude
lat_idx = (np.abs(latitude - lat)).argmin()
# get index for April 1935, e.g. with numpy.where
time_idx = np.where(yyyymm == '193504')

# now extract value from windspeed array 
windspeed[time_idx, lat_idx, lon_idx]

array([[4.6494937]], dtype=float32)

For the last step, this method relies on you, because you need to make sure you know which dimension of the wind speed array is time, latitude and longitude. An alternative way is to use the xarray dataset, where each datapoint is strictly tight to the corresponding time and coordinates: 

In [8]:
# access dimension names and lengths of the dataset: 
dataset.dims

Frozen(SortedKeysDict({'time': 1980, 'bnds': 2, 'lat': 192, 'lon': 288}))

In [9]:
# use xarray.sel function to extratc value by dimension label and value 
gridpoint = dataset.sel(lon=longitude[lon_idx]).sel(lat=latitude[lat_idx]).sel(time= time[time_idx])

display(gridpoint.sfcWind)

<xarray.DataArray 'sfcWind' (time: 1)>
array([4.649494], dtype=float32)
Coordinates:
  * time     (time) object 1935-04-16 00:00:00
    lat      float64 57.96
    lon      float64 12.5
    height   float64 10.0
Attributes:
    standard_name:  wind_speed
    long_name:      Near-Surface Wind Speed
    comment:        near-surface (usually, 10 meters) wind speed.
    units:          m s-1
    original_name:  U10
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    history:        2020-08-26T10:13:32Z altered by CMOR: Treated scalar dime...

## Part 2: NAO from .txt 

For text files, a commonly used data containter is the python package **pandas**. You read in tables as a so-called dataframe, from which it is easy to calculate averages, access information for specific rows and columns and so on. 

Check out the documentation for the function I use, to understand what the input parameters mean: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_table.html

Btw, you have to inspect the data first and try to read in with different options, because it is not always obvious e.g. which lines belong the header is. The NAO text file is properly formatted, but sometimes you need more optional parameters for the function to read in text files with a more complicated structure. 

In [10]:
import pandas as pd 

In [11]:
path = 'nao_station_monthly.txt'
nao = pd.read_table(path, header= 1, delimiter = '\s+' , na_values = -999 )
# inspect table 
nao

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
1865,-0.6,-1.2,0.2,-0.2,-0.4,0.0,0.5,1.5,1.8,-2.0,-0.9,0.8
1866,0.5,0.8,-0.6,-2.3,-2.0,0.9,-0.5,-0.2,2.4,-0.3,-0.5,0.2
1867,-3.5,1.1,-4.3,1.8,-4.2,0.1,-2.0,1.9,1.4,2.2,-3.5,-0.1
1868,0.7,3.0,3.6,1.7,2.3,3.1,0.4,1.5,-2.8,3.5,-1.8,-0.1
1869,1.0,2.5,0.2,-0.2,-2.7,-1.9,-0.3,-1.0,-0.4,-1.4,1.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2016,0.0,2.4,1.4,-1.7,-0.8,-0.2,0.4,0.2,2.8,-1.2,1.0,0.9
2017,-0.4,1.2,1.5,-1.0,-2.4,1.4,1.8,0.3,2.3,0.7,-1.2,0.8
2018,2.4,0.9,-1.0,1.2,3.9,0.0,1.5,2.2,0.6,1.0,-0.2,0.1
2019,0.1,0.1,2.6,0.0,-1.8,-2.5,-0.9,-0.5,1.0,-1.3,0.4,0.8


There are two ways to access values in a **pandas** dataframe. You can either make use of column names and row indices (in this dataframe the years are the row indices) or use traditional numpy array indexing:

In [12]:
# access columns using pandas 
nao['Jan']

1865   -0.6
1866    0.5
1867   -3.5
1868    0.7
1869    1.0
       ... 
2016    0.0
2017   -0.4
2018    2.4
2019    0.1
2020    2.1
Name: Jan, Length: 156, dtype: float64

In [13]:
# use numpy array indexing for pandas format to extract e.g. first column 
nao.iloc[:,0]

1865   -0.6
1866    0.5
1867   -3.5
1868    0.7
1869    1.0
       ... 
2016    0.0
2017   -0.4
2018    2.4
2019    0.1
2020    2.1
Name: Jan, Length: 156, dtype: float64

In [17]:
# get a time array YYYYMM with the same shape as table (156x12)

# create empty array (note the time array is a numpy array, no pandas dataframe)
YYYYMM = np.empty((nao.shape[0],nao.shape[1])).astype(str)
# you need: index values containing the years and column 2-13 which correspond to the months
# loop through columns 
for col in np.arange(nao.shape[1]):
    for row in np.arange(nao.shape[0]):
        year = str(nao.index[row])
        # think again of zero-padding 
        if col < 9: 
            month = '0' +str(col + 1)
        else:
            month = str(col + 1)
        YYYYMM[row,col] = year + month

When and what was the latest non-missing value? **pandas** has an inbuilt function for this:

In [18]:
# get last index value (remember: these are the years here) for each column
pos = nao.apply(lambda x: x.last_valid_index())
pos

Jan    2020
Feb    2020
Mar    2019
Apr    2019
May    2019
Jun    2019
Jul    2019
Aug    2019
Sep    2019
Oct    2019
Nov    2019
Dec    2019
dtype: int64

In [19]:
# get row and column indices for pandas dataframe 
pos_row = pos[pos == pos.max()][0] # maximum index for valid data entries 
pos_col = pos.index[pos == pos.max()][-1] # latest month for maximum index 

# get column index as a number rather than a month string to use for time array (remember:it was a numpy array)
col_idx = nao.columns.get_loc(pos_col)

# get row index as a number rather than the year number 
row_idx = np.abs(nao.index[0] - pos_row)

# what
print('what:',nao.loc[pos_row, pos_col])
#when 
print('when:', YYYYMM[row_idx, col_idx])

what: 3.3
when: 202002


## Part 3: csv files

Juhu!  **pandas** can even be used for .csv files: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html 

Let us have a look, what this mysterious .csv file contains: 

In [209]:
path = 'ex1.csv'
mysteriousdata = pd.read_csv(path)
mysteriousdata

,latitude,longitude
0,89.992765,132.236841
1,88.585873,120.029879
2,87.837700,105.002651
3,88.741973,104.215916
4,86.170943,118.239172
5,78.561500,-1.718100
6,87.432200,93.677800
7,86.655400,115.918400
8,86.152573,116.355934
9,87.144100,114.260600


## Bonus part: 

**1.Easy**

In [342]:
# read in file one by one 


# this time extract directly the variable of interest (we dont need lons and lats multiple times)
var = 'sfcWind'

wind1 = xr.open_dataset('sfcWind_Amon_INM-CM4-8_historical_r1i1p1f1_gr1_185001-194912.nc')[var]
wind2 = xr.open_dataset('sfcWind_Amon_INM-CM4-8_historical_r1i1p1f1_gr1_195001-199912.nc')[var]
wind3 = xr.open_dataset('sfcWind_Amon_INM-CM4-8_historical_r1i1p1f1_gr1_200001-201412.nc')[var]


In [363]:
# append arrays
windspeed = np.append(wind1,wind2 ,axis = 0)
windspeed = np.append(windspeed, wind3, axis = 0)
windspeed.shape

(1980, 120, 180)

**1.Medium**

In [364]:
# same steps as before to read in data 
# merge datarrays in one step by stacking along time dimension (which is the first dimension, axis = 0 )
windspeed = np.vstack([wind1, wind2,wind3])
windspeed.shape

(1980, 120, 180)

**3.Advanced**

In [367]:
# use wild cards to find files that contain the string INM-CM4-8
import glob 

files = glob.glob('*INM-CM4-8*')
files

['sfcWind_Amon_INM-CM4-8_historical_r1i1p1f1_gr1_185001-194912.nc',
 'sfcWind_Amon_INM-CM4-8_historical_r1i1p1f1_gr1_200001-201412.nc',
 'sfcWind_Amon_INM-CM4-8_historical_r1i1p1f1_gr1_195001-199912.nc']

In [371]:
# loop over files to read in 
for f in np.arange(len(files)):
    data = xr.open_dataset(files[f])[var]
    if f == 0: 
        windspeed = data # assign array for first file 
    else:
        windspeed= np.vstack([windspeed, data]) # then concatenate/stack the other files